In [ ]:
# Authors: Dr. Min Wang, Noah Gardner

In [1]:
from pathlib import Path

import numpy as np
import scipy.io as sio
from fcmeans import FCM
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import accuracy_score, cluster
from sklearn.metrics import adjusted_rand_score

In [3]:
from juliacall import Main as jl
jl.seval('include("./datasets.jl")')
jl.seval('ENV["DATADEPS_ALWAYS_ACCEPT"] = "true"')

'true'

In [5]:
# Constants
# number of experiments
noexp = 3

# number of classes
labellist = [2, 5, 10, 11]
L = len(labellist)

In [9]:
ip_data_path = Path(jl.seval('datadep"IndianPinesData"')) / 'Indian_pines.mat'
ip_labels_path = Path(jl.seval('datadep"IndianPinesLabels"')) / 'Indian_pines_gt.mat'
ip_data = sio.loadmat(ip_data_path)['indian_pines']
ip_labels = sio.loadmat(ip_labels_path)['indian_pines_gt']

ip_data = ip_data.reshape(ip_data.shape[0] * ip_data.shape[1], ip_data.shape[2])
ip_labels = ip_labels.reshape(ip_labels.shape[0] * ip_labels.shape[1])

In [10]:
idx = np.where(np.isin(ip_labels, labellist))
ip_data = ip_data[idx]
ip_labels = ip_labels[idx]
print(ip_data.shape)
print(ip_labels.shape)

(5338, 220)
(5338,)


In [12]:
# fit an FCM model and return parameters
def runFCM(X):
    fcm = FCM(n_clusters=L) # random_state=int(np.random.uniform()*100))
    fcm.fit(X)
    labels = fcm.predict(X)
    return fcm._centers, fcm.u, labels

In [19]:
def checkFCM(preds, labels):
    # create a mapping list: class i --> pred j
    c = cluster.contingency_matrix(labels, preds, eps=1, dtype=int)
    remapped_centers = linear_sum_assignment(c, maximize=True)
    
    # need to convert labellist to [0,3]
    remapped_preds = np.array([remapped_centers[1][pred] for pred in preds])
    
    print('new experiment')
    print('prediction frequency count: ', np.unique(preds, return_counts=True))
    print('contigency matrix:\n', c)
    print('re-mapped centers: ', remapped_centers)
    print('accuracy: ', accuracy_score(remapped_preds, labels))

    # for debugging, adjusted rand score will be the same before and after remapping
    print('adjusted rand score: ', adjusted_rand_score(preds, labels))
    print('adjusted rand score (after remap): ', adjusted_rand_score(remapped_preds, labels))
    return accuracy_score(remapped_preds, labels)

In [17]:
C, U, preds = runFCM(ip_data)

In [20]:
checkFCM(preds, ip_labels)

new experiment
prediction frequency count:  (array([0, 1, 2, 3]), array([ 357, 1790,  870, 2321]))
contigency matrix:
 [[   3  782  205  442]
 [ 351    7   71   58]
 [   1  187  249  539]
 [   6  818  349 1286]]
re-mapped centers:  (array([0, 1, 2, 3]), array([1, 0, 2, 3]))
accuracy:  0.03821656050955414
adjusted rand score:  0.11548376621170907
adjusted rand score:  0.11548376621170907


0.03821656050955414